# Quering the *flight phase files*

The *flight segments* are stored in *YAML* files. In order to read these files:

In [1]:
%load_ext autoreload
%autoreload 2

import flightphase as fp
import pandas as pd # only for showing the results in a table


filepath = 'ACLOUD_P5_Flight-Segments_20170608_RF14.yaml'

ds = fp.FlightPhaseFile(filepath)

### Query all segments of a specific kinds

Example with one kind

In [2]:
queried = ds.selectKind(['high_level'])

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts,remark
0,[high_level],high level leg 1,[],ACLOUD_P5_RF14_hl01,2017-06-08 08:08:54,2017-06-08 08:25:57,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}",[],NaN
1,[high_level],high level2,[],ACLOUD_P5_RF14_hl02,2017-06-08 08:36:30,2017-06-08 08:58:26,[9800],"{'GOOD': ['P5_RF14_sonde02'], 'BAD': [], 'UGLY...",[],NaN
2,[high_level],high level 3,[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[],Satellite underflight (A-Train)
3,[high_level],high level 4,[],ACLOUD_P5_RF14_hl04,2017-06-08 09:31:41,2017-06-08 09:51:04,[9800],"{'GOOD': ['P5_RF14_sonde04'], 'BAD': [], 'UGLY...",[],NaN
4,[high_level],high level 5,[],ACLOUD_P5_RF14_hl05,2017-06-08 09:51:33,2017-06-08 10:00:14,[9800],"{'GOOD': ['P5_RF14_sonde05'], 'BAD': [], 'UGLY...",[],NaN
5,[high_level],high level 8,[],ACLOUD_P5_RF14_hl08,2017-06-08 11:11:20,2017-06-08 11:52:57,[9700],"{'GOOD': [], 'BAD': ['P5_RF14_sonde06'], 'UGLY...",NaN,NaN
6,[high_level],high level 9,[],ACLOUD_P5_RF14_hl09,2017-06-08 11:53:38,2017-06-08 12:05:14,[9700],"{'GOOD': ['P5_RF14_sonde07'], 'BAD': [], 'UGLY...",NaN,NaN


 Example with more than one kind

In [3]:
kinds = ['high_level', 'nya_overflight']

queried = ds.selectKind(kinds)

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts,remark
0,"[major_ascend, nya_overflight, low_level_Sveab...",major ascend 1,[whale-watching loop],ACLOUD_P5_RF14_ma1,2017-06-08 07:36:50,2017-06-08 08:08:54,"[100, 9800]","{'GOOD': [], 'BAD': [], 'UGLY': []}",NaN,NaN
1,[high_level],high level leg 1,[],ACLOUD_P5_RF14_hl01,2017-06-08 08:08:54,2017-06-08 08:25:57,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}",[],NaN
2,[high_level],high level2,[],ACLOUD_P5_RF14_hl02,2017-06-08 08:36:30,2017-06-08 08:58:26,[9800],"{'GOOD': ['P5_RF14_sonde02'], 'BAD': [], 'UGLY...",[],NaN
3,[high_level],high level 3,[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[],Satellite underflight (A-Train)
4,[high_level],high level 4,[],ACLOUD_P5_RF14_hl04,2017-06-08 09:31:41,2017-06-08 09:51:04,[9800],"{'GOOD': ['P5_RF14_sonde04'], 'BAD': [], 'UGLY...",[],NaN
5,[high_level],high level 5,[],ACLOUD_P5_RF14_hl05,2017-06-08 09:51:33,2017-06-08 10:00:14,[9800],"{'GOOD': ['P5_RF14_sonde05'], 'BAD': [], 'UGLY...",[],NaN
6,[high_level],high level 8,[],ACLOUD_P5_RF14_hl08,2017-06-08 11:11:20,2017-06-08 11:52:57,[9700],"{'GOOD': [], 'BAD': ['P5_RF14_sonde06'], 'UGLY...",NaN,NaN
7,[high_level],high level 9,[],ACLOUD_P5_RF14_hl09,2017-06-08 11:53:38,2017-06-08 12:05:14,[9700],"{'GOOD': ['P5_RF14_sonde07'], 'BAD': [], 'UGLY...",NaN,NaN
8,"[cross_pattern, nya_overflight]",cross pattern 2,[],ACLOUD_P5_RF14_cp02,2017-06-08 12:05:14,2017-06-08 12:25:40,[9700],"{'GOOD': [], 'BAD': [], 'UGLY': []}","[{'kinds': ['cross_pattern_leg'], 'name': 'cro...",NaN


### Sorting out all segments given a list of kinds

In [4]:
sort_out = ['short_turn', 'holding_pattern', 'major_ascend', 'major_descend']

queried = ds.selectKind(sort_out, invertSelection=True)

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts,remark
0,[high_level],high level leg 1,[],ACLOUD_P5_RF14_hl01,2017-06-08 08:08:54,2017-06-08 08:25:57,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}",[],NaN
1,[high_level],high level2,[],ACLOUD_P5_RF14_hl02,2017-06-08 08:36:30,2017-06-08 08:58:26,[9800],"{'GOOD': ['P5_RF14_sonde02'], 'BAD': [], 'UGLY...",[],NaN
2,[high_level],high level 3,[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[],Satellite underflight (A-Train)
3,[high_level],high level 4,[],ACLOUD_P5_RF14_hl04,2017-06-08 09:31:41,2017-06-08 09:51:04,[9800],"{'GOOD': ['P5_RF14_sonde04'], 'BAD': [], 'UGLY...",[],NaN
4,[high_level],high level 5,[],ACLOUD_P5_RF14_hl05,2017-06-08 09:51:33,2017-06-08 10:00:14,[9800],"{'GOOD': ['P5_RF14_sonde05'], 'BAD': [], 'UGLY...",[],NaN
5,"[cross_pattern, polarstern_overflight]",cross pattern 1,[],ACLOUD_P5_RF14_cp01,2017-06-08 10:00:14,2017-06-08 10:10:51,[9800],NaN,"[{'kinds': ['cross_pattern_leg'], 'name': 'cro...",NaN
6,[large_descend],large descend,[not along straight line],ACLOUD_P5_RF14_ld01,2017-06-08 10:10:51,2017-06-08 10:22:44,"[9800, 200]","{'GOOD': [], 'BAD': [], 'UGLY': []}",NaN,NaN
7,[racetrack pattern],racetrack pattern 1,[],ACLOUD_P5_RF14_rt01,2017-06-08 10:22:44,2017-06-08 11:02:02,"[200, 400, 1500, 2400]","{'GOOD': [], 'BAD': [], 'UGLY': []}","[{'kinds': ['racetrack_leg'], 'name': 'racetra...",NaN
8,[large_ascend],large ascend,[],ACLOUD_P5_RF14_la01,2017-06-08 11:02:25,2017-06-08 11:11:20,"[2400, 9700]","{'GOOD': [], 'BAD': [], 'UGLY': []}",NaN,NaN
9,[high_level],high level 8,[],ACLOUD_P5_RF14_hl08,2017-06-08 11:11:20,2017-06-08 11:52:57,[9700],"{'GOOD': [], 'BAD': ['P5_RF14_sonde06'], 'UGLY...",NaN,NaN


### Query a segment with a specific value in a specific attribute

#### Example 1: Select element with specific name

In [5]:
attribute = 'name'
value = 'racetrack pattern 1'

queried = ds.select(attribute, value) 

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts
0,[racetrack pattern],racetrack pattern 1,[],ACLOUD_P5_RF14_rt01,2017-06-08 10:22:44,2017-06-08 11:02:02,"[200, 400, 1500, 2400]","{'GOOD': [], 'BAD': [], 'UGLY': []}","[{'kinds': ['racetrack_leg'], 'name': 'racetra..."


Also with inverted selection

In [6]:
queried = ds.select(attribute, value, invertSelection=True) 

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts,remark
0,"[major_ascend, nya_overflight, low_level_Sveab...",major ascend 1,[whale-watching loop],ACLOUD_P5_RF14_ma1,2017-06-08 07:36:50,2017-06-08 08:08:54,"[100, 9800]","{'GOOD': [], 'BAD': [], 'UGLY': []}",NaN,NaN
1,[high_level],high level leg 1,[],ACLOUD_P5_RF14_hl01,2017-06-08 08:08:54,2017-06-08 08:25:57,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}",[],NaN
2,[holding_pattern],holding pattern 1,[],ACLOUD_P5_RF14_ho01,2017-06-08 08:25:57,2017-06-08 08:36:30,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}","[{'kinds': ['short_turn'], 'start': 2017-06-08...",NaN
3,[high_level],high level2,[],ACLOUD_P5_RF14_hl02,2017-06-08 08:36:30,2017-06-08 08:58:26,[9800],"{'GOOD': ['P5_RF14_sonde02'], 'BAD': [], 'UGLY...",[],NaN
4,[short_turn],NaN,NaN,NaN,2017-06-08 08:58:26,2017-06-08 08:59:08,NaN,NaN,NaN,NaN
5,[high_level],high level 3,[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[],Satellite underflight (A-Train)
6,[short_turn],NaN,NaN,NaN,2017-06-08 09:30:30,2017-06-08 09:31:41,NaN,NaN,NaN,NaN
7,[high_level],high level 4,[],ACLOUD_P5_RF14_hl04,2017-06-08 09:31:41,2017-06-08 09:51:04,[9800],"{'GOOD': ['P5_RF14_sonde04'], 'BAD': [], 'UGLY...",[],NaN
8,[short_turn],NaN,NaN,NaN,2017-06-08 09:51:04,2017-06-08 09:51:33,NaN,NaN,NaN,NaN
9,[high_level],high level 5,[],ACLOUD_P5_RF14_hl05,2017-06-08 09:51:33,2017-06-08 10:00:14,[9800],"{'GOOD': ['P5_RF14_sonde05'], 'BAD': [], 'UGLY...",[],NaN


#### Example 2: Specific remark

In [7]:
attribute = 'remark'
value = 'Satellite underflight (A-Train)'

queried = ds.select(attribute, value)

df = pd.DataFrame(queried)
df

,kinds,name,remark,irregularities,segment_id,start,end,levels,dropsondes,parts
0,[high_level],high level 3,Satellite underflight (A-Train),[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[]


#### Example 3: Specific level

In [8]:
attribute = 'levels'
value = 9800

queried = ds.select(attribute, value)

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts,remark
0,"[major_ascend, nya_overflight, low_level_Sveab...",major ascend 1,[whale-watching loop],ACLOUD_P5_RF14_ma1,2017-06-08 07:36:50,2017-06-08 08:08:54,"[100, 9800]","{'GOOD': [], 'BAD': [], 'UGLY': []}",NaN,NaN
1,[high_level],high level leg 1,[],ACLOUD_P5_RF14_hl01,2017-06-08 08:08:54,2017-06-08 08:25:57,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}",[],NaN
2,[holding_pattern],holding pattern 1,[],ACLOUD_P5_RF14_ho01,2017-06-08 08:25:57,2017-06-08 08:36:30,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}","[{'kinds': ['short_turn'], 'start': 2017-06-08...",NaN
3,[high_level],high level2,[],ACLOUD_P5_RF14_hl02,2017-06-08 08:36:30,2017-06-08 08:58:26,[9800],"{'GOOD': ['P5_RF14_sonde02'], 'BAD': [], 'UGLY...",[],NaN
4,[high_level],high level 3,[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[],Satellite underflight (A-Train)
5,[high_level],high level 4,[],ACLOUD_P5_RF14_hl04,2017-06-08 09:31:41,2017-06-08 09:51:04,[9800],"{'GOOD': ['P5_RF14_sonde04'], 'BAD': [], 'UGLY...",[],NaN
6,[high_level],high level 5,[],ACLOUD_P5_RF14_hl05,2017-06-08 09:51:33,2017-06-08 10:00:14,[9800],"{'GOOD': ['P5_RF14_sonde05'], 'BAD': [], 'UGLY...",[],NaN
7,"[cross_pattern, polarstern_overflight]",cross pattern 1,[],ACLOUD_P5_RF14_cp01,2017-06-08 10:00:14,2017-06-08 10:10:51,[9800],NaN,"[{'kinds': ['cross_pattern_leg'], 'name': 'cro...",NaN
8,[large_descend],large descend,[not along straight line],ACLOUD_P5_RF14_ld01,2017-06-08 10:10:51,2017-06-08 10:22:44,"[9800, 200]","{'GOOD': [], 'BAD': [], 'UGLY': []}",NaN,NaN


But what if we want only a specific level without ascend and descend? We can require that the attribute value is strictly equal to the one, that we give, i.e. thare no other values:

In [9]:
queried = ds.select(attribute, value, strict=True)

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts,remark
0,[high_level],high level leg 1,[],ACLOUD_P5_RF14_hl01,2017-06-08 08:08:54,2017-06-08 08:25:57,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}",[],NaN
1,[holding_pattern],holding pattern 1,[],ACLOUD_P5_RF14_ho01,2017-06-08 08:25:57,2017-06-08 08:36:30,[9800],"{'GOOD': [], 'BAD': [], 'UGLY': []}","[{'kinds': ['short_turn'], 'start': 2017-06-08...",NaN
2,[high_level],high level2,[],ACLOUD_P5_RF14_hl02,2017-06-08 08:36:30,2017-06-08 08:58:26,[9800],"{'GOOD': ['P5_RF14_sonde02'], 'BAD': [], 'UGLY...",[],NaN
3,[high_level],high level 3,[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[],Satellite underflight (A-Train)
4,[high_level],high level 4,[],ACLOUD_P5_RF14_hl04,2017-06-08 09:31:41,2017-06-08 09:51:04,[9800],"{'GOOD': ['P5_RF14_sonde04'], 'BAD': [], 'UGLY...",[],NaN
5,[high_level],high level 5,[],ACLOUD_P5_RF14_hl05,2017-06-08 09:51:33,2017-06-08 10:00:14,[9800],"{'GOOD': ['P5_RF14_sonde05'], 'BAD': [], 'UGLY...",[],NaN
6,"[cross_pattern, polarstern_overflight]",cross pattern 1,[],ACLOUD_P5_RF14_cp01,2017-06-08 10:00:14,2017-06-08 10:10:51,[9800],NaN,"[{'kinds': ['cross_pattern_leg'], 'name': 'cro...",NaN


This functions only if the attribute has more values, as for level or kind.

### Query all segments with good (or bad) dropsondes

In [10]:
queried = ds.selectDropSondes('GOOD')

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes,parts,remark
0,[high_level],high level2,[],ACLOUD_P5_RF14_hl02,2017-06-08 08:36:30,2017-06-08 08:58:26,[9800],"{'GOOD': ['P5_RF14_sonde02'], 'BAD': [], 'UGLY...",[],NaN
1,[high_level],high level 3,[],ACLOUD_P5_RF14_hl03,2017-06-08 08:59:08,2017-06-08 09:30:30,[9800],"{'GOOD': ['P5_RF14_sonde03'], 'BAD': [], 'UGLY...",[],Satellite underflight (A-Train)
2,[high_level],high level 4,[],ACLOUD_P5_RF14_hl04,2017-06-08 09:31:41,2017-06-08 09:51:04,[9800],"{'GOOD': ['P5_RF14_sonde04'], 'BAD': [], 'UGLY...",[],NaN
3,[high_level],high level 5,[],ACLOUD_P5_RF14_hl05,2017-06-08 09:51:33,2017-06-08 10:00:14,[9800],"{'GOOD': ['P5_RF14_sonde05'], 'BAD': [], 'UGLY...",[],NaN
4,[high_level],high level 9,[],ACLOUD_P5_RF14_hl09,2017-06-08 11:53:38,2017-06-08 12:05:14,[9700],"{'GOOD': ['P5_RF14_sonde07'], 'BAD': [], 'UGLY...",NaN,NaN


In [11]:
queried = ds.selectDropSondes('BAD')

df = pd.DataFrame(queried)
df

,kinds,name,irregularities,segment_id,start,end,levels,dropsondes
0,[high_level],high level 8,[],ACLOUD_P5_RF14_hl08,2017-06-08 11:11:20,2017-06-08 11:52:57,[9700],"{'GOOD': [], 'BAD': ['P5_RF14_sonde06'], 'UGLY..."


In [12]:
queried = ds.selectDropSondes('UGLY')

df = pd.DataFrame(queried)
df

""
